[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nmickevicius/MCW_BIOP_03-238_MRI/blob/main/04_linear_algebra/linear_algebra.ipynb)


## 0) Notation & Quick Complex Refresh

- Scalars: real $a$, complex $z = a + ib$; $i^2 = -1$.  
- Conjugate: $z^* = a - ib$; magnitude $|z|=\sqrt{a^2+b^2}$.  
- Vectors $\mathbf{x}\in \mathbb{C}^N$, matrices $A\in\mathbb{C}^{m\times n}$.  
- Conjugate transpose: $A^H = (A^*)^T$.  
- In MRI, raw k-space data and many operators are **complex-valued**, so conjugation and phases matter.


In [3]:

# Interactive: Complex number and its conjugate on the complex plane
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np 
from ipywidgets import interact, IntSlider, FloatSlider
from IPython.display import clear_output, HTML, display
import os

def plot_complex(a=1.0, b=1.0):
    z = a + 1j*b
    zc = np.conj(z)
    fig = plt.figure(figsize=(5,5))
    ax = plt.gca()
    ax.axhline(0, lw=1)
    ax.axvline(0, lw=1)
    ax.scatter([z.real], [z.imag], s=100, label='z = a + ib')
    ax.scatter([zc.real], [zc.imag], s=100, marker='x', label='conj(z) = a - ib')
    ax.set_xlabel('Real')
    ax.set_ylabel('Imag')
    r = max(2, abs(a)+1, abs(b)+1)
    ax.set_xlim(-r, r); ax.set_ylim(-r, r); ax.set_aspect('equal', 'box')
    ax.legend(loc='upper right')
    ax.set_title(f"|z| = {np.abs(z):.3f}")
    plt.show()

interact(plot_complex, a=FloatSlider(min=-5, max=5, step=0.1, value=1.0),
                     b=FloatSlider(min=-5, max=5, step=0.1, value=1.0));


interactive(children=(FloatSlider(value=1.0, description='a', max=5.0, min=-5.0), FloatSlider(value=1.0, descr…


## 1) Vectors, Inner Products, and Norms — Notes

- Inner product (complex): $\langle \mathbf{a},\mathbf{b}\rangle = \mathbf{a}^H \mathbf{b}$.  
- Norm: $\|\mathbf{a}\|_2 = \sqrt{\langle \mathbf{a},\mathbf{a}\rangle}$.  
- Orthogonality: $\langle \mathbf{a},\mathbf{b}\rangle = 0$.  



### 1) Worked Example (step-by-step)

Let $\mathbf{a}=\begin{bmatrix}1+i\\ 2-i\end{bmatrix}$, $\mathbf{b}=\begin{bmatrix}2-i\\ -i\end{bmatrix}$.  
1) $\mathbf{a}^H=[\,1-i\;\;2+i\,]$.  
2) $\langle \mathbf{a},\mathbf{b}\rangle = (1-i)(2-i) + (2+i)(-i) = (1-3i)+(1-2i)=2-5i$.  
3) $\|\mathbf{a}\|^2 = (1-i)(1+i)+(2+i)(2-i)=2+5=7\Rightarrow \|\mathbf{a}\|=\sqrt{7}$.


## Matrix Multiplication & Rotation Matrices

Consider the (m x n) matrix A and the (n x p) matrix B:
$$
\mathbf{A} =
\begin{bmatrix}
a_{11} & a_{12} & \dots & a_{1n} \\
a_{21} & a_{22} & \dots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \dots & a_{mn} \\
\end{bmatrix}
$$
$$
\mathbf{B} =
\begin{bmatrix}
b_{11} & b_{12} & \dots & b_{1p} \\
b_{21} & b_{22} & \dots & b_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
b_{n1} & b_{n2} & \dots & b_{np} \\
\end{bmatrix}
$$

The multiplication of $\mathbf{AB}$ will be a (m x p) matrix $\mathbf{C}$ with the following formula for each entry

$$
c_{ij} = \sum_{k=1}^n a_{ik}b_{kj}
$$

- **Dimensions** must match between columns of A and rows of B: if $A\in\mathbb{C}^{m\times n}$ and $B\in\mathbb{C}^{n\times p}$, then $AB\in\mathbb{C}^{m\times p}$. 
- **Associativity:** $(AB)C = A(BC)$ (but in general $AB \neq BA$).  


## Rotation Matrices

**Rotation matrices** are matrices used to rotate a vector while preserving its length. In imaging (including MRI geometry gradients & coordinate transforms), rotations re-orient coordinates without changing magnitudes.

Common 3D rotations (right-handed, angles \(\alpha,\beta,\gamma\) about \(x,y,z\), respectively):
$$
R_x(\alpha) = \begin{bmatrix}
1 & 0 & 0\\
0 & \cos\alpha & -\sin\alpha\\
0 & \sin\alpha & \cos\alpha
\end{bmatrix},\quad
R_y(\beta) = \begin{bmatrix}
\cos\beta & 0 & \sin\beta\\
0 & 1 & 0\\
-\sin\beta & 0 & \cos\beta
\end{bmatrix},\quad
R_z(\gamma) = \begin{bmatrix}
\cos\gamma & -\sin\gamma & 0\\
\sin\gamma & \cos\gamma & 0\\
0 & 0 & 1
\end{bmatrix}.
$$

Composition (apply $R_x$ then $R_y$ then $R_z$ to a vector $\mathbf{v}$):  
$$
\mathbf{v}_{\text{rot}} = R_z(\gamma)\,R_y(\beta)\,R_x(\alpha)\,\mathbf{v}.
$$

### 1B) Worked Example (step-by-step, 2D rotation)

Rotate $\mathbf{v}=\begin{bmatrix}1\\0\end{bmatrix}$ by angle $\theta$ in 2D.  
Use $R(\theta) = \begin{bmatrix}\cos\theta & -\sin\theta\\ \sin\theta & \cos\theta\end{bmatrix}$.

1) Multiply:
$$
R(\theta)\mathbf{v} =
\begin{bmatrix}\cos\theta & -\sin\theta\\ \sin\theta & \cos\theta\end{bmatrix}
\begin{bmatrix}1\\0\end{bmatrix}
= \begin{bmatrix}\cos\theta \\ \sin\theta\end{bmatrix}.
$$

In [4]:
# Interactive: Rotate a vector on the unit sphere in 3D and show its x–y "shadow" (projection)
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider
from IPython.display import clear_output, HTML, display
import os

def Rx(a):
    ca, sa = np.cos(a), np.sin(a)
    return np.array([[1,0,0],[0,ca,-sa],[0,sa,ca]], dtype=float)

def Ry(b):
    cb, sb = np.cos(b), np.sin(b)
    return np.array([[cb,0,sb],[0,1,0],[-sb,0,cb]], dtype=float)

def Rz(c):
    cc, sc = np.cos(c), np.sin(c)
    return np.array([[cc,-sc,0],[sc,cc,0],[0,0,1]], dtype=float)

def rot_demo_with_shadow(theta_x=0.0, theta_y=0.0, theta_z=0.0, v_theta=45.0, v_phi=30.0):
    # Base unit vector from spherical angles v_theta (polar from +z) and v_phi (azimuth in x-y), degrees
    th = np.deg2rad(v_theta)
    ph = np.deg2rad(v_phi)
    v0 = np.array([np.sin(th)*np.cos(ph), np.sin(th)*np.sin(ph), np.cos(th)], dtype=float)

    # Rotation angles (degrees -> radians)
    ax = np.deg2rad(theta_x)
    ay = np.deg2rad(theta_y)
    az = np.deg2rad(theta_z)

    # Compose rotation: apply Rx, then Ry, then Rz
    R = Rz(az) @ Ry(ay) @ Rx(ax)
    v = R @ v0
    v = v / (np.linalg.norm(v) + 1e-12)  # keep numerically unit length

    # x–y plane projection ("shadow")
    p = np.array([v[0], v[1], 0.0])

    # Unit sphere wireframe
    u = np.linspace(0, 2*np.pi, 40)
    t = np.linspace(0, np.pi, 20)
    xs = np.outer(np.cos(u), np.sin(t))
    ys = np.outer(np.sin(u), np.sin(t))
    zs = np.outer(np.ones_like(u), np.cos(t))

    # x–y unit circle (intersection of sphere with plane z=0)
    circle_u = np.linspace(0, 2*np.pi, 200)
    circle_x = np.cos(circle_u)
    circle_y = np.sin(circle_u)
    circle_z = np.zeros_like(circle_u)

    fig = plt.figure(figsize=(6,6))
    ax3d = fig.add_subplot(111, projection='3d')

    # Sphere grid
    ax3d.plot_wireframe(xs, ys, zs, linewidth=0.3, alpha=0.5)

    # Draw the x–y plane as a thin surface for visual context
    plane_x = np.linspace(-1.2, 1.2, 2)
    plane_y = np.linspace(-1.2, 1.2, 2)
    PX, PY = np.meshgrid(plane_x, plane_y)
    PZ = np.zeros_like(PX)
    ax3d.plot_surface(PX, PY, PZ, alpha=0.08)

    # Unit circle on x–y plane
    ax3d.plot(circle_x, circle_y, circle_z, linewidth=1.0)

    # Original and rotated vectors
    ax3d.quiver(0,0,0, v0[0],v0[1],v0[2], length=1.0, normalize=False)
    ax3d.quiver(0,0,0, v[0],v[1],v[2], length=1.0, normalize=False)

    # Shadow vector on x–y plane
    ax3d.quiver(0,0,0, p[0],p[1],p[2], length=1.0, normalize=False)

    # Drop line from tip of rotated vector to its projection on the x–y plane
    ax3d.plot([v[0], p[0]], [v[1], p[1]], [v[2], p[2]], linestyle='--', linewidth=1.0)

    # Axes & labels
    lim = 1.2
    ax3d.set_xlim([-lim, lim]); ax3d.set_ylim([-lim, lim]); ax3d.set_zlim([-lim, lim])
    ax3d.set_xlabel('x'); ax3d.set_ylabel('y'); ax3d.set_zlabel('z')
    ax3d.set_title("3D Rotation with x–y Plane Projection (Shadow)")
    ax3d.view_init(elev=22, azim=35)
    plt.show()

interact(rot_demo_with_shadow,
         theta_x=FloatSlider(min=-180,max=180,step=1,value=0, description='θx'),
         theta_y=FloatSlider(min=-180,max=180,step=1,value=0, description='θy'),
         theta_z=FloatSlider(min=-180,max=180,step=1,value=0, description='θz'),
         v_theta=FloatSlider(min=0,max=180,step=1,value=45, description='vθ'),
         v_phi=FloatSlider(min=-180,max=180,step=1,value=30, description='vφ'));


interactive(children=(FloatSlider(value=0.0, description='θx', max=180.0, min=-180.0, step=1.0), FloatSlider(v…


## 2) Linear Systems $A\mathbf{x}=\mathbf{b}$

- Solve small systems by elimination or factorization.  
- In MRI, linear encoding/reconstruction often reduces to solving such systems (per pixel/voxel or globally).



### 2) Worked Example (step-by-step)

$$
A=\begin{bmatrix}3&1\\ 1&2\end{bmatrix},\ 
\mathbf{b}=\begin{bmatrix}9\\ 8\end{bmatrix}
\Rightarrow \mathbf{x}=\begin{bmatrix}2\\3\end{bmatrix}.
$$

Row2 $\leftarrow$ Row2 $-\frac{1}{3}$ Row1 leads to $\frac{5}{3}x_2=5\Rightarrow x_2=3$.  
Back-substitute: $3x_1+3=9\Rightarrow x_1=2$.


In [5]:

# Interactive: 2x2 system as intersection of lines
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np 
from ipywidgets import interact, IntSlider, FloatSlider
from IPython.display import clear_output, HTML, display
import os

def solve_and_plot(a11=3.0, a12=1.0, a21=1.0, a22=2.0, b1=9.0, b2=8.0):
    A = np.array([[a11,a12],[a21,a22]], dtype=float)
    b = np.array([b1,b2], dtype=float)
    x = None
    if abs(np.linalg.det(A)) > 1e-9:
        x = np.linalg.solve(A,b)

    xx = np.linspace(-5,5,400)
    # lines: a11*x + a12*y = b1 ; a21*x + a22*y = b2
    y1 = (b1 - a11*xx)/a12 if abs(a12)>1e-9 else np.full_like(xx, np.nan)
    y2 = (b2 - a21*xx)/a22 if abs(a22)>1e-9 else np.full_like(xx, np.nan)

    plt.figure(figsize=(6,6))
    plt.plot(xx, y1, label='Eqn 1')
    plt.plot(xx, y2, label='Eqn 2')
    if x is not None and np.isfinite(x).all():
        plt.scatter([x[0]],[x[1]], s=80, label=f"Solution ({x[0]:.2f},{x[1]:.2f})")
    plt.xlim(-5,5); plt.ylim(-5,5)
    plt.axhline(0,color='k',lw=0.5); plt.axvline(0,color='k',lw=0.5)
    plt.legend(); plt.title("Intersection = solution")
    plt.gca().set_aspect('equal','box')
    plt.show()

interact(solve_and_plot,
         a11=FloatSlider(min=-5,max=5,step=0.1,value=3.0),
         a12=FloatSlider(min=-5,max=5,step=0.1,value=1.0),
         a21=FloatSlider(min=-5,max=5,step=0.1,value=1.0),
         a22=FloatSlider(min=-5,max=5,step=0.1,value=2.0),
         b1=FloatSlider(min=-10,max=10,step=0.1,value=9.0),
         b2=FloatSlider(min=-10,max=10,step=0.1,value=8.0));


interactive(children=(FloatSlider(value=3.0, description='a11', max=5.0, min=-5.0), FloatSlider(value=1.0, des…


## 3) Least Squares & Projections — Notes

- Overdetermined $A\mathbf{x}\approx \mathbf{b}$.  
- Normal equations: $A^H A \hat{\mathbf{x}} = A^H \mathbf{b}$.  
- Solution: $(A^H A)^{-1}A^H\mathbf{b}$



### 3) Worked Example (Elimination)

$$
A=\begin{bmatrix}1&0\\ 1&1\\ 0&1\end{bmatrix},\ 
\mathbf{b}=\begin{bmatrix}1\\2\\1\end{bmatrix}
\Rightarrow \hat{\mathbf{x}}=\begin{bmatrix}1\\1\end{bmatrix}.
$$

Compute $A^TA=\begin{bmatrix}2&1\\1&2\end{bmatrix}$, $A^Tb=\begin{bmatrix}3\\3\end{bmatrix}$ and solve. 

2 $\times$ Row1 - Row2:

$(4 - 1)x_1 + (2 - 2)x_2 = (6-3)$

$3 x_1 = 3 \Rightarrow x_1 = 1$

Back substitute (Row2):

$1 + 2 x_2 = 3$

$x_2 = 1$



### 3b) Matrix Inverse in Least Squares (step-by-step, 2×2 case)

Sometimes it’s helpful to **see** the normal-equation solution written with a matrix inverse:
$$
\hat{\mathbf{x}} = (A^T A)^{-1} A^T \mathbf{b}.
$$
> We usually **avoid** forming an inverse explicitly (for numerical stability), but by hand it’s instructive.

For our example,
$$
A=\begin{bmatrix}1&0\\ 1&1\\ 0&1\end{bmatrix},\quad 
A^T A=\begin{bmatrix}2&1\\ 1&2\end{bmatrix},\quad 
A^T\mathbf{b}=\begin{bmatrix}3\\3\end{bmatrix}.
$$

**Goal:** compute $(A^T A)^{-1}$ by hand and then $\hat{\mathbf{x}}=(A^T A)^{-1}A^T\mathbf{b}$.

#### Step 1 — Determinant
For a $2\times2$ matrix $\begin{bmatrix}a&b\\ c&d\end{bmatrix}$,
$\det = ad - bc$.
Here, $a=d=2$, $b=c=1$, so
$$
\det(A^T A) = 2\cdot 2 - 1\cdot 1 = 4 - 1 = 3.
$$

#### Step 2 — Adjugate (a.k.a. adjoint)
$$
\operatorname{adj}\!\left(\begin{bmatrix}a&b\\ c&d\end{bmatrix}\right) =
\begin{bmatrix}d&-\,b\\ -\,c&a\end{bmatrix}.
$$
Thus,
$$
\operatorname{adj}(A^T A) = \begin{bmatrix}2&-1\\ -1&2\end{bmatrix}.
$$

#### Step 3 — Inverse
$$
(A^T A)^{-1} = \frac{1}{\det(A^T A)}\,\operatorname{adj}(A^T A)
= \frac{1}{3}\begin{bmatrix}2&-1\\ -1&2\end{bmatrix}.
$$

#### Step 4 — Multiply by $A^T\mathbf{b}$
$$
\hat{\mathbf{x}}=(A^T A)^{-1}A^T\mathbf{b} 
= \frac{1}{3}\begin{bmatrix}2&-1\\ -1&2\end{bmatrix}\begin{bmatrix}3\\3\end{bmatrix}
= \frac{1}{3}\begin{bmatrix}6-3\\ -3+6\end{bmatrix}
= \frac{1}{3}\begin{bmatrix}3\\3\end{bmatrix}
= \begin{bmatrix}1\\1\end{bmatrix}.
$$

In [6]:

# Interactive: Least squares line fit y = ax + b to noisy points (projection intuition)
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np 
from ipywidgets import interact, IntSlider, FloatSlider
from IPython.display import clear_output, HTML, display
import os

def lsq_demo(n=6, noise=0.3, true_a=1.0, true_b=0.0, seed=0):
    rng = np.random.default_rng(int(seed))
    x = np.linspace(-1,1,n)
    y = true_a*x + true_b + noise*rng.standard_normal(n)
    # A = [x, 1]; solve (A^T A) theta = A^T y
    A = np.c_[x, np.ones_like(x)]
    ATA = A.T @ A
    ATy = A.T @ y
    theta = np.linalg.solve(ATA, ATy)
    a_hat, b_hat = theta

    xx = np.linspace(-1.2,1.2,200)
    yy = a_hat*xx + b_hat

    plt.figure(figsize=(6,4))
    plt.scatter(x,y,label='data')
    plt.plot(xx,yy,label=f'LS fit: y={a_hat:.2f}x+{b_hat:.2f}')
    plt.ylim(-3,3)
    plt.title("Least Squares")
    plt.legend(); plt.show()

interact(lsq_demo,
         n=IntSlider(min=3,max=20,step=1,value=6),
         noise=FloatSlider(min=0.0,max=1.0,step=0.05,value=0.3),
         true_a=FloatSlider(min=-2,max=2,step=0.1,value=1.0),
         true_b=FloatSlider(min=-1,max=1,step=0.1,value=0.0),
         seed=IntSlider(min=0,max=999,step=1,value=0));


interactive(children=(IntSlider(value=6, description='n', max=20, min=3), FloatSlider(value=0.3, description='…


## 4) Fourier Transform as a Linear Operator — Notes

- Discrete Fourier Transform (DFT): $X[k]=\sum_{n=0}^{N-1} x[n]e^{-i2\pi kn/N}$.  
- Inverse: $x[n]=\frac{1}{N}\sum_{k=0}^{N-1} X[k]e^{i2\pi kn/N}$.  
- In MRI, the **DFT matrix** maps images \(\leftrightarrow\) k-space.

Construct the DFT matrix as follows:

$$
\mathbf{F} = 
\begin{bmatrix}
e^{-i 2 \pi \cdot 0 \cdot 0 /N} & e^{-i 2 \pi \cdot 0 \cdot 1 /N} & \dots & e^{-i 2 \pi \cdot 0 \cdot (N-1) /N} \\
e^{-i 2 \pi \cdot 1 \cdot 0 /N} & e^{-i 2 \pi \cdot 1 \cdot 1 /N} & \dots & e^{-i 2 \pi \cdot 1 \cdot (N-1) /N} \\
\vdots & \vdots & \ddots & \vdots \\
e^{-i 2 \pi \cdot (N-1) \cdot 0 /N} & e^{-i 2 \pi \cdot (N-1) \cdot 1 /N} & \dots & e^{-i 2 \pi \cdot (N-1) \cdot (N-1) /N} 
\end{bmatrix}
$$

The DFT of a vector $\mathbf{x}$ is then given by the matrix-vector product:
$$
\mathbf{X} = \mathbf{Fx}
$$


In [7]:

# Interactive: Build a length-N signal and inspect DFT magnitude/phase
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np 
from ipywidgets import interact, IntSlider, FloatSlider
from IPython.display import clear_output, HTML, display
import os

def dft_demo(N=8, idx1=0, val1=1.0, idx2=4, val2=1.0):
    N = int(N)
    x = np.zeros(N, dtype=complex)
    idx1 = int(np.clip(idx1, 0, N-1))
    idx2 = int(np.clip(idx2, 0, N-1))
    x[idx1] = val1
    x[idx2] = x[idx2] + val2
    k = np.arange(N)
    n = np.arange(N)[:,None]
    W = np.exp(-1j*2*np.pi*n*k/N)
    X = (W.T @ x).T  # DFT via matrix multiply
    fig = plt.figure(figsize=(8,4))
    plt.subplot(1,2,1); plt.stem(np.arange(N), np.real(x)); plt.title("Signal (Real)"); plt.ylim(0, 2.1)
    plt.subplot(1,2,2); plt.stem(np.arange(N), np.abs(X)); plt.title("|DFT|"); plt.ylim(0, 4.1)
    plt.tight_layout(); plt.show()

interact(dft_demo,
         N=IntSlider(min=4,max=32,step=1,value=32),
         idx1=IntSlider(min=0,max=31,step=1,value=0),
         val1=FloatSlider(min=0,max=2,step=0.1,value=1.5),
         idx2=IntSlider(min=0,max=31,step=1,value=2),
         val2=FloatSlider(min=0,max=2,step=0.1,value=1.0));


interactive(children=(IntSlider(value=32, description='N', max=32, min=4), IntSlider(value=0, description='idx…


## 6) Basic Regularization (Tikhonov/Ridge) 

## Skipped in lecture -- interesting, but not required learning 

Solve $\min_{\mathbf{x}} \|A\mathbf{x}-\mathbf{b}\|^2 + \lambda \|\mathbf{x}\|^2$  
$\Rightarrow (A^H A + \lambda I)\hat{\mathbf{x}} = A^H \mathbf{b}$.  
Increasing $\lambda$ stabilizes solutions with noise, but biases the solution vector toward zero.


In [8]:

# Interactive: Ridge regression path for y = ax + b
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np 
from ipywidgets import interact, IntSlider, FloatSlider
from IPython.display import clear_output, HTML, display
import os

def ridge_demo(n=8, noise=0.4, true_a=1.0, true_b=0.0, lam=0.5, seed=0):
    rng = np.random.default_rng(int(seed))
    x = np.linspace(-1,1,n)
    y = true_a*x + true_b + noise*rng.standard_normal(n)
    A = np.c_[x, np.ones_like(x)]
    ATA = A.T @ A
    ATy = A.T @ y
    theta = np.linalg.solve(ATA + lam*np.eye(2), ATy)
    a_hat, b_hat = theta

    xx = np.linspace(-1.2,1.2,200)
    yy = a_hat*xx + b_hat

    plt.figure(figsize=(6,4))
    plt.scatter(x,y,label='data')
    plt.plot(xx,yy,label=f'Ridge fit (λ={lam:.2f})')
    plt.title("Tikhonov Regularization")
    plt.legend(); plt.show()

interact(ridge_demo,
         n=IntSlider(min=4,max=40,step=1,value=8),
         noise=FloatSlider(min=0.0,max=1.0,step=0.05,value=0.4),
         true_a=FloatSlider(min=-2,max=2,step=0.1,value=1.0),
         true_b=FloatSlider(min=-1,max=1,step=0.1,value=0.0),
         lam=FloatSlider(min=0.0,max=5.0,step=0.05,value=0.5),
         seed=IntSlider(min=0,max=999,step=1,value=0));


interactive(children=(IntSlider(value=8, description='n', max=40, min=4), FloatSlider(value=0.4, description='…